In [3]:
import pandas as pd
import numpy as np
import torch

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt



from transformerclass import *


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cuda


In [11]:
def load_dataset(dataset_path):
    """Loads a dataset from a CSV file."""
    return pd.read_csv(dataset_path)

def filter_dataset_by_cases_and_channels(dataset, cases, channels):
    """Filters a dataset to keep only the rows that correspond to the specified cases and channels."""
    selected_rows = pd.DataFrame()
    for case_number in cases:
        rows_for_case = dataset[dataset['Case'] == f'case{case_number}']
        selected_rows = pd.concat([selected_rows, rows_for_case])
    selected_rows = selected_rows[selected_rows['Channel'].isin(channels)]
    return selected_rows

def split_dataset_for_train_val_test(data_pd):
    """Splits a dataset into three parts: train, validation, and test."""
    normal_data = data_pd[data_pd['norm/ab'] == 'normal']
    abnormal_data = data_pd[data_pd['norm/ab'] == 'abnormal']
    
    # We only need normal data for training, but validation and test need both normal and abnormal data.
    train_data, intermediate_data = train_test_split(normal_data, test_size=0.2, shuffle=True)
    validation_data, test_data = train_test_split(pd.concat([abnormal_data, intermediate_data]), test_size=0.8, shuffle=True)

    return train_data, validation_data, test_data

def train_val_test(dataset_path=None, cases=[], channels=[]):
    """Loads a dataset, filters it, and splits it for training, validation, and test."""
    dataset = load_dataset(dataset_path)
    filtered_dataset = filter_dataset_by_cases_and_channels(dataset, cases, channels)
    train_data, validation_data, test_data = split_dataset_for_train_val_test(filtered_dataset)
    return train_data, validation_data, test_data


datapath = r'C:\Users\brech\THESIS_local\ToyADMOS\ToycarCSV.csv'
cases = [1]
channels = ['ch1']
train_data, validation_data, test_data = train_val_test(dataset_path=datapath, cases=cases, channels=channels)


In [12]:
print("train shape: ", train_data.shape, "\n", "validation shape: ", validation_data.shape, "\n", "test_dataset: ", test_data.shape)

train shape:  (1080, 7) 
 validation shape:  (106, 7) 
 test_dataset:  (428, 7)


In [ ]:
#helper functions 
def find_path_to_wav(full_sample_name):
    for root, dirs, files in os.walk(os.path.dirname(datapath)):
        for name in files:
            if name == full_sample_name:
                path_to_wavFile = os.path.abspath(os.path.join(root, name))
                return path_to_wavFile


def get_sample_waveform_normalised(full_sample_name, start = 0, stop = 11):
    #returns waveform values, cut to seconds going from start to stop
    sample_path = find_path_to_wav(full_sample_name)
    waveform, sample_rate = librosa.load(sample_path, sr= None)
    waveform = waveform[int(start*sample_rate): int(stop*sample_rate)]
        
    return librosa.util.normalize(waveform)